# Описание проекта.
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Шаг 1. Откройте таблицу и изучите общую информацию о данных

In [95]:
import pandas as pd
data = pd.read_csv(r'C:\Users\Natalia\Downloads\data (2).csv')
data.info()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


# Вывод
В выборке 21525 наблюдений: 5 текстовых, 5 целочисленных, 2 вещественных.
В столбцах days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) пропущенно 2174 наблюдений.


# Шаг 2. Предобработка данных

1. Обработка пропусков 

In [51]:
print('Столбец: total_income')
#обработаю пропуски в столбце total_income
#определю какую долю составляют пропущенные данные 
data['total_income'].value_counts() #показывает сколько данных пропущенно
print('Колличество пропущеных значений:', len(data[data['total_income'].isnull()]))
print('Какой % от данных составляет:',  100 -((19351 * 100) / 21525 ), '%')
#появление пропусков в данных возможно возникло из-за человеческого фактора
#в этом столбце пропущенны значения колличественного типа, поэтому будет разумно заполнить пропуски медианным значением по 
#данному столбцу
data_median = data['total_income'].median()
print('Медиана:', data_median)
data['total_income'] = data['total_income'].fillna(value = data_median) # замена пропусков на медиану
print('Проверим сколько пропусков осталось после обработки:', len(data[data['total_income'].isnull()]))
print(data.head())

Столбец: total_income
Колличество пропущеных значений: 2174
Какой % от данных составляет: 10.099883855981417 %
Медиана: 145017.93753253992
Проверим сколько пропусков осталось после обработки: 0
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  граждан

3. Замена типов данных

In [52]:
data['total_income'] = data['total_income'].astype('int64')
data['dob_years'] = data['dob_years'].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Вывод: Переменные типа float64: children, dob_years представим в виде целых чисел int64, так как это количество детей и возраст клиента. 
Столбец days_employed тоже лучше представить в виде int64, но пока не знаем, что делать с этим столбцом, поэтому оставим его в таком виде.

4. Обработка дубликатов и некорректных значений

In [60]:
print('Обработанные значения для каждого столбца')
print()
dubl = data['education'].value_counts()
data['education'] = data['education'].str.lower() #приведем столбец к нижнему регистру
print(data['education'].value_counts())
print()
print(data['family_status'].value_counts())
data['family_status'] = data['family_status'].str.lower() #приведем столбец к нижнему регистру
print()
print(data['income_type'].value_counts())
print()
print('Удаление неккоретного значения пола(gender)')
data = data[data['gender'] != 'XNA']
print(data['gender'].value_counts())
print()
print('Удаление дубликатов')
print(data.duplicated().sum())
data = data.drop_duplicates() 
print(data.duplicated().sum())
print("Проверка корректности данных children")
children_median = data['children'].median() 
print()
data.loc[(data['children'] == -1) | (data['children'] == 20), 'children'] = children_median #замена некорр.знач. на медиану
print("проверка корректности данных children после обработки")
print(data['children'].value_counts())
print()
print("Проверка корректности данных dob_years")
#Посмотрим количество строк в датафрейме, где возраст меньше 18 лет(минимальны возраст заёмщика)
data.loc[data['dob_years'] < 18, 'dob_years'].count()
#Посмотрим, являются строки с возрастом менее 18 лет - нулём. Для этого посчитаем кол-во строк меньше 18 лет и не равных 0.
data.loc[(data['dob_years'] <= 18) & (data['dob_years'] != 0), 'dob_years'].count()
#Заполним пропуски полученными значениями
for i in data['income_type'].unique():
    mean_value = data.loc[data['income_type'] == i, 'dob_years'].mean()
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == i), 'dob_years'] = mean_value
data.loc[data['dob_years'] == 0, 'dob_years'].count()
print('Минимальный возраст:', data['dob_years'].min())
print('Максимальный возраст:',data['dob_years'].max())

Обработанные уникальные значения для каждого столбца

среднее                15172
высшее                  5250
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12339
гражданский брак          4150
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

сотрудник          11084
компаньон           5077
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Удаление неккоретного значения пола(gender)
F    14174
M     7279
Name: gender, dtype: int64

Удаление дубликатов
0
0
Проверка корректности данных children

проверка корректности данных children после обработки
0    14213
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Проверка корректности данных dob_years
М

5. Создание 2-х новых дататфрейма 

In [39]:
log_education = data[['education', 'education_id']]
print(log_education)
log_family_status = data[['family_status', 'family_status_id']]
print(log_family_status)

      education  education_id
0        высшее             0
1       среднее             1
2       среднее             1
3       среднее             1
4       среднее             1
...         ...           ...
21520   среднее             1
21521   среднее             1
21522   среднее             1
21523   среднее             1
21524   среднее             1

[21453 rows x 2 columns]
          family_status  family_status_id
0       женат / замужем                 0
1       женат / замужем                 0
2       женат / замужем                 0
3       женат / замужем                 0
4      гражданский брак                 1
...                 ...               ...
21520  гражданский брак                 1
21521   женат / замужем                 0
21522  гражданский брак                 1
21523   женат / замужем                 0
21524   женат / замужем                 0

[21453 rows x 2 columns]


In [40]:
print('Удалю столбцы education и family_status ') #можно оставить только их идентификаторы
data.drop('education', axis= 1 , inplace= True)
data.drop('family_status', axis= 1 , inplace= True)
print(data.info())

Удалю столбцы education и family_status 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education_id      21453 non-null  int64  
 4   family_status_id  21453 non-null  int64  
 5   gender            21453 non-null  object 
 6   income_type       21453 non-null  object 
 7   debt              21453 non-null  int64  
 8   total_income      21453 non-null  int64  
 9   purpose           21453 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 1.8+ MB
None


6. Создание столбца "total_income_category" 

In [99]:
def create(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    elif 30001 <= total_income <= 50000:
        return 'D'
    elif 50001 <= total_income <= 200000:
        return 'C'
    elif 200001 <= total_income <= 1000000:
        return 'B'
    elif total_income >= 1000001:
        return 'A'
data['total_income_category'] = data['total_income'].apply(create)
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

7. Создание функции 'purpose_category'

In [103]:
print(data['purpose'].value_counts())
def categori(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'авто' in purpose:
        return 'операции с автомобилем'
    elif ('жиль' in purpose) | ('недвижим' in purpose) :
        return 'операции с недвижимостью'
    elif 'образов' in purpose:
        return 'получение образования'
data['purpose_category'] = data['purpose'].apply(categori)
print(data.head(10))
    


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

# Вывод

In [48]:
print('Как влияет колличество детей и семейное положение на возврат кредита в срок?')
data_pivot = data.pivot_table(index = ['children'], columns = 'family_status_id',
values = 'debt', aggfunc = 'mean')
# index - столбец или столбцы, по которым происходит группировка данных
# columns - столбец по значениям которого будет происходить группировка
# values - значения, по которым мы хотим увидеть сводную таблицу
# aggfunc - функция, которая будет применяться к значениям
print(data_pivot)
print()
print('0', 'женат/замужем')
print('1', 'гражданский брак')
print('2', 'вдовец / вдова')
print('3', 'в разводе')
print('4', 'Не женат / не замужем')
print('В результате работы выявленно, что ')
#ничего не понятно, сложно сделать какие-то выводы

Есть ли зависимость между количеством детей и возвратом кредита в срок?
family_status_id         0         1         2         3         4
children                                                          
0                 0.068911  0.084487  0.061988  0.070886  0.092707
1                 0.082689  0.118593  0.090909  0.067308  0.115813
2                 0.094586  0.087464  0.150000  0.086420  0.120000
3                 0.068273  0.142857  0.000000  0.090909  0.125000
4                 0.103448  0.000000  0.000000  0.000000  0.500000
5                 0.000000  0.000000       NaN       NaN       NaN

0 женат/замужем
1 гражданский брак
2 вдовец / вдова
3 в разводе
4 Не женат / не замужем


In [87]:
#Разобьем заемщиков по категориям по колличеству детей
def children_new(children):
    if 1 <= children <= 2:
        return 'есть дети'
    if children >= 3:
        return 'многодетный'
    return 'нет детей'
data['children_new'] = data['children'].apply(children_new)
print('Как влияет колличество детей и семейное положение на возврат кредита в срок?')
data_pivot = data.pivot_table(index = ['children_new'], columns = 'debt',
values = 'gender', aggfunc = 'count')
# Создаем столбцы
data_pivot.columns = ['no_debt', 'debt']
# Считаем долю должников
data_pivot['share_debtors'] = data_pivot['debt'] / (data_pivot['debt'] + data_pivot['no_debt'])
print(data_pivot)
print()
print('Вывод: Мы видим, что в семьях, где есть дети, уровень задолжностей выше. Однако в многодетных семьях уровень задолжностей ниже, чем в семьях с 1,2 детьми. Меньше всего задолжностей в семьях, где дети отсутствуют.')

Как влияет колличество детей и семейное положение на возврат кредита в срок?
              no_debt  debt  share_debtors
children_new                              
есть дети        6235   638       0.092827
многодетный       417    39       0.085526
нет детей       13132  1064       0.074951

Мы видим, что в семьях, где есть дети, уровень задолжностей выше. Однако в многодетных семьях уровень задолжностей ниже, чем в семьях с 1,2 детьми. Меньше всего задолжностей в семьях, где дети отсутствуют.


In [96]:
print('Есть ли зависимость между семейным положением и возвратом кредита в срок?')
data_pivot = data.pivot_table(index = ['family_status'], columns = 'debt',
values = 'gender', aggfunc = 'count')
# Создаем столбцы
data_pivot.columns = ['no_debt', 'debt']
# Считаем долю должников
data_pivot['share_debtors'] = data_pivot['debt'] / (data_pivot['debt'] + data_pivot['no_debt'])
print(data_pivot)
print("Вывод: Больше задолжностей имеют люди, находящиеся в гражданском браке или люди не состоящие в браке")

Есть ли зависимость между семейным положением и возвратом кредита в срок?
                       no_debt  debt  share_debtors
family_status                                      
Не женат / не замужем     2539   274       0.097405
в разводе                 1110    85       0.071130
вдовец / вдова             897    63       0.065625
гражданский брак          3789   388       0.092890
женат / замужем          11449   931       0.075202
Вывод: Больше задолжностей имеют люди, находящиеся в гражданском браке или люди не состоящие в браке


In [101]:
print('Есть ли зависимость между уровнем дохода и возвратом кредита в срок?')
data_pivot = data.pivot_table(index = ['total_income_category'], columns = 'debt',
values = 'gender', aggfunc = 'count')
# Создаем столбцы
data_pivot.columns = ['no_debt', 'debt']
# Считаем долю должников
data_pivot['share_debtors'] = data_pivot['debt'] / (data_pivot['debt'] + data_pivot['no_debt'])
print(data_pivot)
print("Вывод: Больше всего имеют задолжности люди, которые имеют средний доход(С), меньше всего люди с доходом категории(D)")

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
                       no_debt  debt  share_debtors
total_income_category                              
A                           23     2       0.080000
B                         4685   356       0.070621
C                        12723  1190       0.085532
D                          329    21       0.060000
E                           20     2       0.090909
Вывод: Больше всего имеют задолжности люди, которые имеют средний доход(С)


In [105]:
print('Как разные цели кредита влияют на его возврат в срок?')
data_pivot = data.pivot_table(index = ['purpose_category'], columns = 'debt',
values = 'gender', aggfunc = 'count')
# Создаем столбцы
data_pivot.columns = ['no_debt', 'debt']
# Считаем долю должников
data_pivot['share_debtors'] = data_pivot['debt'] / (data_pivot['debt'] + data_pivot['no_debt'])
print(data_pivot)
print("Вывод: Больше всего имеют задолжности люди, берущие кредиты на получение образования и операции с автомобилем")

Как разные цели кредита влияют на его возврат в срок?
                          no_debt  debt  share_debtors
purpose_category                                      
операции с автомобилем       3912   403       0.093395
операции с недвижимостью    10058   782       0.072140
получение образования        3652   370       0.091994
проведение свадьбы           2162   186       0.079216
Вывод: Больше всего имеют задолжности люди, берущие кредиты на получение образования и операции с автомобилем
